## Instance Struture

In [2]:
mutable struct Instance
    n :: Int64
    D
    l
    avg :: Float64
    function Instance(s)
        D = zeros(1,1)
        nNodes = 0
        l = 0
        avg = 0
        if s != "STDIN"
            open(s) do f
                line = 1
                lineContent = readline(f)
                lineSplited = split(lineContent, " ")
                nNodes = parse(Int64, lineSplited[1])
                l = parse(Int64, lineSplited[2])
                D = zeros(nNodes,nNodes)
                avgRatio = 1/((nNodes^2)-nNodes)

                while ! eof(f)
                    lineContent = readline(f)
                    lineSplited = split(lineContent, " ")
                    for col = 1:nNodes
                        value = parse(Float64, lineSplited[col])
                        D[line,col] = value
                        avg += value*avgRatio
                    end

                    line += 1
                end
            end
        else
            line = 1
            lineContent = readline("STDIN")
            lineSplited = split(lineContent, " ")
            nNodes = parse(Int64, lineSplited[1])
            l = parse(Int64, lineSplited[2])
            D = zeros(nNodes,nNodes)
            avgRatio = 1/((nNodes^2)-nNodes)

            while ! eof(f)
                lineContent = readline(f)
                lineSplited = split(lineContent, " ")
                for col = 1:nNodes
                    value = parse(Float64, lineSplited[col])
                    D[line,col] = value
                    avg += value*avgRatio
                end

                line += 1
            end
        end
        new(nNodes,D,l,avg)
    end
end

## Solution Structure


In [2]:
mutable struct Solution
    π :: Array{Int64,1} ## solução: uma permutação de cidades
    v :: Float64        ## valor da solução: distância total
    Solution(s::Solution) = new(copy(s.π),s.v)
    Solution(π,v) = new(copy(π),v)
end
Base.copy(s::Solution) = Solution(s)

 ## Nearest Neighbor

In [3]:
function nearestNeighbor(I::Instance)
    s = Solution([1],0.0)
    free=trues(I.n)
    free[1]=false
    for k=2:I.n
      f=findall(free)
      j=f[findmin(I.D[s.π[end],f])[2]]
      push!(s.π,j)
      free[j]=false
    end
    for lookahead=1:I.l
        for i=1:I.n
            s.v += I.D[s.π[i],s.π[((i+lookahead-1)%(I.n))+1]]
        end
    end
    s
end

nearestNeighbor (generic function with 1 method)

## Random Nearest Neighbor

In [4]:
function randomNearestNeighbor(I::Instance,k)
    s = Solution([1],0.0)
    free=trues(I.n)
    free[1]=false
    for i=2:I.n
        f=findall(free)
        ncand=min(k,I.n-i+1)
        j=f[sortperm(I.D[s.π[end],f])[rand(1:ncand)]]
        push!(s.π,j)
        free[j]=false
    end
    for lookahead=1:I.l
        for i=1:I.n
            s.v += I.D[s.π[i],s.π[((i+lookahead-1)%(I.n))+1]]
        end
    end
    s
end

randomNearestNeighbor (generic function with 1 method)

## Local Search with Adjacent Cities Swap

In [5]:
function calcIndex(index, n)
    result = 0
    if index <= 0
        result = n-(abs(index)%n)
    else
        result = (((index)-1)%(n))+1
    end
    result
end

function localSearchAdjacentCitiesSwap(I::Instance, s₀::Solution)
    s = copy(s₀)
    i=1
    v=0.0
    while i<I.n
        i=2
        while i<I.n
            
            
            v = s.v
#             println(i)
#             println(s.π)
            for l1=1:I.l+2
                index1 = calcIndex((i+1)-l1+1, I.n)
                for l2=1:I.l
                    index2 = calcIndex(index1+l2, I.n)
#                     println("$(index1) - $(index2)")
                    v -= I.D[s.π[index1],s.π[index2]]
                end
            end
            
            
            s.π[i],s.π[i+1] = s.π[i+1],s.π[i] # swap
            
#             println(s.π)
            for l1=1:I.l+2
                index1 = calcIndex((i+1)-l1+1, I.n)
                for l2=1:I.l
                    index2 = calcIndex(index1+l2, I.n)
                    v += I.D[s.π[index1],s.π[index2]]
                end
            end
            
#             v=s.v-I.D[s.π[i-1],s.π[i]]-I.D[s.π[i+1],s.π[i+2]]+I.D[s.π[i-1],s.π[i+1]]+I.D[s.π[i],s.π[i+2]]
#             -I.D[s.π[i-1],s.π[i]]
#             -I.D[s.π[i+1],s.π[i+2]]
#             +I.D[s.π[i-1],s.π[i+1]]
#             +I.D[s.π[i],s.π[i+2]]
            
            
#             vok = 0
#             for lookahead=1:I.l # calc new value with pi modified
#                 for i=1:I.n
#                     vok += I.D[s.π[i],s.π[((i+lookahead-1)%(I.n))+1]]
#                 end
#             end
#             println("v: $(v) | vok: $(vok) | s.v: $(s.v)")
            
            s.π[i],s.π[i+1] = s.π[i+1],s.π[i] # swap again
            
            v<s.v && break ## first improvement
            i += 1
        end
        if i<I.n
#             println("Old v: $(s.v) New v: $(v)")
            s.π[i],s.π[i+1] = s.π[i+1],s.π[i]
            s.v = v
        end
    end
    s
end

localSearchAdjacentCitiesSwap (generic function with 1 method)

In [ ]:
I = Instance("instances/rml10.txt")
b = nearestNeighbor(I)
println("Nearest Neighbor: $(b.v).")

nIterations = 100 # DEFINIR ITERAÇÕES!
k = 2 # DEFINIR K!

println("Local Search Adjacent Cities Swap")
for i = 1:nIterations
    s = randomNearestNeighbor(I,k) 
    s = localSearchAdjacentCitiesSwap(I,s)
    if s.v < b.v
        b = copy(s)
    end
    println("Iteration $i: $(b.v)\n")
end
println("Best solution $(b.v).")



Nearest Neighbor: 3.0128831e7.
Local Search Adjacent Cities Swap

In [6]:
I = Instance("instances/rml9.txt")
solution = [1, 34, 35, 36, 2, 37, 3, 38, 72, 4, 39, 5, 73, 40, 74, 41, 6, 7, 8, 42, 9, 10, 75, 43, 11, 44, 12, 45, 77, 46, 78, 47, 48, 15, 16, 49, 17, 50, 51, 18, 84, 19, 52, 20, 85, 53, 381, 21, 54, 86, 382, 55, 87, 22, 378, 88, 23, 56, 89, 24, 57, 25, 90, 58, 91, 26, 59, 27, 60, 92, 61, 28, 93, 29, 62, 30, 63, 31, 376, 32, 64, 33, 94, 377, 95, 65, 101, 96, 97, 436, 379, 98, 380, 383, 384, 113, 112, 125, 124, 134, 111, 147, 123, 133, 146, 158, 169, 182, 197, 196, 207, 206, 195, 404, 217, 205, 194, 409, 157, 168, 181, 408, 145, 193, 233, 225, 216, 204, 192, 144, 132, 391, 122, 121, 385, 131, 120, 109, 388, 390, 142, 130, 439, 108, 119, 83, 82, 81, 100, 80, 14, 79, 99, 106, 105, 140, 117, 128, 139, 153, 163, 176, 393, 188, 250, 200, 213, 222, 415, 249, 230, 248, 212, 247, 246, 229, 221, 211, 245, 244, 403, 243, 242, 241, 187, 399, 396, 401, 186, 400, 174, 173, 152, 185, 162, 151, 161, 150, 149, 136, 126, 114, 137, 127, 386, 103, 441, 102, 115, 104, 389, 387, 66, 442, 67, 68, 69, 70, 71, 116, 392, 138, 175, 76, 13, 164, 107, 397, 118, 129, 141, 154, 165, 189, 201, 402, 202, 155, 190, 166, 178, 191, 398, 395, 394, 179, 143, 167, 110, 180, 159, 135, 170, 148, 183, 160, 171, 184, 199, 210, 220, 226, 411, 410, 414, 237, 265, 420, 437, 416, 275, 268, 423, 272, 264, 236, 263, 262, 413, 261, 422, 419, 260, 259, 258, 257, 256, 255, 156, 412, 203, 177, 215, 224, 214, 232, 223, 231, 254, 253, 252, 251, 418, 208, 417, 218, 198, 209, 219, 438, 305, 306, 307, 427, 338, 337, 336, 335, 334, 333, 332, 432, 375, 374, 304, 373, 372, 331, 371, 370, 330, 369, 368, 345, 367, 329, 303, 328, 364, 327, 431, 302, 363, 326, 301, 362, 325, 300, 344, 299, 429, 361, 324, 298, 360, 297, 323, 430, 322, 296, 321, 295, 320, 294, 293, 319, 318, 317, 291, 316, 290, 315, 314, 288, 351, 313, 350, 340, 287, 312, 349, 286, 433, 311, 348, 285, 339, 310, 284, 309, 283, 308, 280, 440, 426, 267, 271, 405, 274, 270, 277, 238, 276, 273, 269, 239, 281, 282, 279, 266, 234, 227, 240, 406, 228, 428, 341, 342, 346, 347, 352, 343, 353, 354, 355, 434, 356, 357, 289, 424, 358, 421, 292, 425, 435, 359, 365, 366, 278, 235, 407, 172] 

vok = 0
for lookahead=1:I.l 
    for i=1:I.n
        vok += I.D[solution[i],solution[((i+lookahead-1)%(I.n))+1]]
    end
end
println("vok: $(vok) ")

vok: 5.833692e6 
